titulos # - subtitulos  ##

# ***<p style="text-align:center;">Análise de Texto através de Bases de Dados Vetoriais</p>***

explicar o porque de utilizar o  chromaDB neste projeto, -> explicar a o processo de pesquisa e assim

bibliotecas 

### importação biblioteca para usar o sistemas de base de dados Chromadb

In [ ]:
import chromadb
from chromadb.utils import embedding_functions

importação de funcionalidades cridas fazer a leitura de arquivos a ser usados durante este projeto

In [ ]:
import leitor

Importação de bibliotecas necessárias para obter a data e hora com o objetivo fututo de realizar o "benchmarking" dos diversos modelos de conversão de embeddings  

In [ ]:
import time 
from datetime import datetime
import os

Criar o cliente para o banco de dados ChromaDB

In [ ]:
chroma_client = chromadb.Client()

: 

Escolher o modelo de embeddings, para a conversão do material fornecido pelo Utizador nos respetivos ficheiros "data" em embeddings vetoriais

In [ ]:
embedder = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

Neste ponto podemos criar a coleção de embeddings numa estrutura de dados predefinida do ChromaDB.

Uma coleção no ChromaDB é um agrupamento com nome de embeddings, dos seus documentos correspondentes e dos respetivos metadados, onde ocorre o processi de organizar, gerir e, de forma crucial, efetuar pesquisas por similaridade em dados vetorizados

No caso do ChromaDB, uma coleção é o local fundamental onde se armazenam e organizam os embeddings vetoriais, os dados originais que estes representam, no caso do nosso projeto lidamos com texto, mas também é possivel em, por exemplo generico, imagens ou outros tipos de dados, e os metadados associados.

collection = chroma_client.get_or_create_collection(
        name="my_collection",
        embedding_function=embedder, # modelo de embedding
        metadata={
        "description": "my first Chroma collection",
        "created": str(datetime.now())
        } 
    )